In [5]:
from qiskit.opflow import X, Z, Y, I
from qiskit.algorithms.optimizers import ADAM, CG, COBYLA, L_BFGS_B, GradientDescent, NELDER_MEAD, \
                                            NFT, POWELL, SLSQP, SPSA, TNC

In [6]:
#Builds a Pauli term by defining which pauli you want at what ndx and the size of the term
def buildMonomial(pauli, ndx, size):
    result = pauli if ndx == 0 else I
    for i in range(1, size):
        result ^= pauli if i == ndx else I
    return result

def buildTerm(size, pauli, coeff = 1.0):
    result = pauli.get(0) or I
    for i in range(1, size):
        result ^= pauli.get(i) or I
    return result * coeff
    
#converts Coupling Matrix -> PauliSumOp
def buildHamiltonian(coupling_mat):
    result = None
    paulis = [X, Y, Z]
    size = len(coupling_mat) // 3
    
    dim = len(coupling_mat.shape)
    num_entries = np.prod(coupling_mat.shape)
    
    for i in range(num_entries):
        f = lambda i,j : i // ((size*3)**j)
        loc = tuple([(f(i,j) % (size*3)) for j in range(dim)])
        
        if coupling_mat[loc] != 0 and tuple(np.sort(loc)) == loc:
            pauli_map = [paulis[aisle // size] for aisle in loc]
            ndx_map = [aisle % size for aisle in loc]
            ndx_pauli = dict(zip(ndx_map, pauli_map))
            term = buildTerm(size, ndx_pauli, coupling_mat[loc])
            result = result + term if result else term
    return result

In [7]:
# Wrapper for Coupling Matrix to make some things easier
# Values are not split across diagonal
class CouplingMatrix:
    
    def __init__(self, N, I_coeff = 0):
        self.qubits = N
        self.size = 3 * N
        self.matrix = np.zeros((self.size, self.size), dtype=float)
        self.I_coeff = I_coeff
        self.set_submatrices()
        self.constant = 0
        
    
    def __setitem__(self, key, value):
        self.matrix[key] = value
        self.matrix[key[::-1]] = value
        self.set_submatrices()
        
    def __getitem__(self, key):
        return self.matrix[key]
    
    def __str__(self):
        return self.matrix.__str__()
    
    def __repr__(self):
        return self.matrix.__str__()
    
    def getHamiltonian(self):
        hamiltonian = buildHamiltonian(self.matrix)
        if hamiltonian:
            return hamiltonian + buildMonomial(I, 0, self.qubits) * self.I_coeff
    
    def setConstant(self, value):
        self.constant = value
    
    def constant(self):
        return self.constant
    
    def set_submatrices(self):
        self.X = self.matrix[:self.qubits,:self.qubits]
        self.Y = self.matrix[self.qubits:2*self.qubits,self.qubits:2*self.qubits]
        self.Z = self.matrix[2*self.qubits:,2*self.qubits:]

In [ ]:
#Builds Schwinger Model Coupling Matrices
def makeSchwinger(N, H_E_coeff, H_M_coeff, H_I_coeff, theta = 0):
    assert N >= 2, "N has to be greater than or equal to 2"
    assert N <= 22, "N can be no greater than 12"
    assert N % 2 == 0, "N must be even"
    
    #Constant
    I_coeff = (1/4) * ((N-1) * (theta/np.pi)**2 + np.ceil((N-1)/2.0) * (1 + 2*(theta/np.pi)) + ((N*(N-1))/2)) * H_E_coeff

    mixer_coupling = CouplingMatrix(N)    
    target_coupling = CouplingMatrix(N, I_coeff)

    #H_E
    for j in range(N-1):
        for k in range(j):
            target_coupling[j + 2*N, k + 2*N] += (N - j - 1)/2.0 * H_E_coeff


    for j in range(N):
        target_coupling[j + 2*N,j + 2*N] += (1/2.0) * ((theta/np.pi) * (N-j-1) + (np.ceil((N-j - 1)/2) - ((N *j)%2) )) * H_E_coeff

    #H_M
    for i in range(N):
        target_coupling[i + 2*N,i + 2*N] += ((-1) ** i) * H_M_coeff

    #H_I
    for i in range(N-1):
        mixer_coupling[i,i + 1] += H_I_coeff
        mixer_coupling[i + N,i + 1 + N] += H_I_coeff
        
    return target_coupling, mixer_coupling

In [8]:
#Sets up optimizers
all_optimizers = [ADAM, CG, COBYLA, L_BFGS_B, GradientDescent, NELDER_MEAD, NFT, POWELL, SLSQP, SPSA, TNC]
def setOptimizers(optimizers, max_iters):
    all_optimizers = [ADAM, CG, COBYLA, L_BFGS_B, GradientDescent, \
                  NELDER_MEAD, NFT, POWELL, SLSQP, SPSA, TNC]
    
    bool_optimizers = [o in optimizers for o in all_optimizers]
    
    for i, optimizer in enumerate(all_optimizers):
        if bool_optimizers[i]:
            print(f"\x1b[32m{optimizer.__name__:15} ON")
        else: 
            print(f"\x1b[31m{optimizer.__name__:15} OFF")
            
    optimizers = [o(maxiter=max_iters) for o in optimizers]
        
    return optimizers

# Find Dimensionless Energy

Given the first three eigenvalues of the Schwinger Model
$$\omega_0, \omega_\_, \omega_+$$

We find the dimensionless Eigenvalues

$$\begin{align}
    f_0 &= \frac{\omega_0}{2Nx} \\
    f_\_ &=  \frac{\omega_\_ - \omega_0}{2\sqrt(x)} - 2\frac{m}{g} \\
    f_+ &= \frac{\omega_+ - \omega_0}{2\sqrt(x)} - 2\frac{m}{g}
\end{align}$$

In [2]:
#Calculates the lowest two eigenvalues, w0 and w1
def findDimensionlessEigen(x, N, m):
    N = int(N)
    m = m
    target_coupling, mixer_coupling = makeSchwinger(N, 1, np.sqrt(x) / 2 * m, x/2.0)

    mixer_hamiltonian = mixer_coupling.getHamiltonian()
    target_hamiltonian = target_coupling.getHamiltonian()

    complete_hamiltonian = mixer_hamiltonian + target_hamiltonian

    matrix_ham = complete_hamiltonian.to_matrix(massive = True)
    sparse_matrix_ham = scipy.sparse.bsr_matrix(matrix_ham)
    eigenvalues = np.sort(scipy.sparse.linalg.eigsh(sparse_matrix_ham, return_eigenvectors=False, which = 'SA', k = 3))
    
    w0, w1, w2 = eigenvalues[:3]
    f0, f1, f2 = w0/(2*N*x), ((w1-w0)/(2*np.sqrt(x))) - 2*m, ((w2-w0)/(2*np.sqrt(x))) - 2*m
    
    return f0, f1, f2